# IN2110 obligatorisk innlevering 1b

Oppgaven har to deler, logistisk regresjon for å klassifisere språk basert på IPA lydskrift, og Named Entity Recognition med HMM. Det er en god idé å lese gjennom hele oppgavesettet før du setter i gang. 

Dersom du har spørsmål så kan du:

* gå på gruppetime,
* spørre på Discourse
* eller sende epost til in2110-hjelp@ifi.uio.no dersom alternativene over av en eller annen grunn ikke passer for spørsmålet ditt.

### Oppsett
Når du har klonet dette github-repoet som denne notebooken ligger i, har du tilgang til datene og hjelpefilene som ligger i denne mappa. Hvis du ønsker å kopiere denne mappa, "1b", over til et annet sted, så skulle det gå bra. Bare pass på at du følger med på om det er oppdateringer her i repoet som gir ut obligen. Når du har aktivert in2110-miljøet med conda, så har du tilgang til pakkene som trengs for å kjøre denne notebooken. Vi har forberedt en notebook med all prekoden, der du også kan fylle ut din kode og kjøre prosessene.

### Innlevering

Innleveringen skal helst bestå av denne Jupyter notebook fylt ut med både kode og tilhørende forklaringer. Vi understreker at innlevering av koden alene __ikke er nok__ for å bestå oppgaven -- vi forventer at notebooken også skal inneholde _beskrivelser_ (på norsk eller engelsk) av hva dere har gjort og _begrunnelser_ for valgene dere har tatt underveis. Bruk helst hele setninger, og matematiske formler om nødvendig. Evalueringstallene bør presenteres i tabeller. Det å  forklare med egne ord (samt begreper vi har gått gjennom på forelesningene) hva dere har implementert og reflektere over hvorvidt løsningen dere har lagt  besvarer oppgaven er en viktig del av læringsprosessen -- ta det på alvor! 

# Del 1: Logistisk regresjon


I den første delen av innleveringen skal vi bruke logistisk regresjon til å utvikle en enkel _språkidentifikator_, dvs. et lite system som skal predikere hvilket språk et ord hører til. Mer spesifikk skal systemet ta ordets _fonetiske transkripsjon_ som input og returnere _navnet på språket_ som ordet mest sannsynlig tilhører.  Systemet skal for eksempel kunne ta transkripsjonen [bʊndɛsvɛɾfaszʊŋ] som input og returnere ``tysk''. 

## Data

For å trene modellen (dvs. finne ut verdier for vektene og skjæringspunktene basert på data) skal vi ta i bruk eksisterende lister over ord med deres fonetiske transkripsjon i såkalt _IPA_-format (IPA står for _[International Phonetic Alphabet](https://upload.wikimedia.org/wikipedia/commons/8/8e/IPA_chart_2018.pdf)_).  Dere trenger ikke å kunne lese eller skrive slike fonetiske transkripsjoner i denne oppgaven. Det viktigste er å forstå at disse transkripsjonene beskriver språklydene som utgjør ordet, samt andre egenskaper slik som lengde, tone, trykk og intonasjon. Fonetiske transkripsjoner forteller oss hvordan et ord bør uttales.

In [21]:
import oblig1b_utils 
train_data, test_data = oblig1b_utils.extract_wordlist()

Reading cached file from ./langid_data.csv


Treningsett: 689238 eksempler, testsett: 76583 eksempler


Data vi skal bruke er lagt i to `DataFrame` tabeller: et treningsett med 90 % av ordene og et testsett med de resterende 10 %. Ordene fra de forskjellige språkene er blandet i disse to tabellene. `DataFrame` er en datastruktur fra Python-biblioteket [`pandas`](https://pandas.pydata.org) og representerer en slags tabell med kolonner og rader. Biblioteket `pandas` gjør det lett å visualisere og manipulere slike tabeller:

In [22]:
print("Statistikk over språkene i treningsett:")
print(train_data.språk.value_counts())
print("Første 30 ord:")
print(train_data[:30])

Statistikk over språkene i treningsett:
språk
koreansk        45051
spansk          45031
vietnamesisk    45025
rumensk         45022
japansk         45020
finsk           45012
kantonesisk     45009
tysk            45001
fransk          44989
engelsk         44961
islandsk        44959
arabisk         44947
swahilisk       43537
mandarin        40357
malayisk        25298
svensk          18971
norsk            9136
farsi            7275
khmer            2945
patwa            1692
Name: count, dtype: int64
Første 30 ord:
                     ord                  IPA         språk
618012          Urheimat       ˈuːɐ̯ˌhaɪ̯maːt          tysk
568297                扫荡         sɑʊ˨˩˦ tɑŋ˥˩      mandarin
616785           lứa đôi          lɯə˩˧ doj˧˥  vietnamesisk
197624           berdebu              bərdəbu      malayisk
88181        convenciste          kombenθiste        spansk
198963               숫돼지       sʰut̚t͈wɛ̝d͡ʑi      koreansk
288725            points             pˈɔ‍ɪnts       e

## Klassen

Her er skjelettet for `LanguageIdentifier`-klassen  som skal implementeres:

In [23]:
import sklearn.linear_model
class LanguageIdentifier:
    """Logistisk regresjonsmodell som tar IPA transkripsjoner av ord som input, 
    og predikerer hvilke språkene disse ordene hører til."""
    
    def __init__(self):
        """Initialiser modellen"""      
        # selve regresjonsmodellen (som brukes all CPU-er på maskinen for trening)
        self.model = sklearn.linear_model.LogisticRegression(solver="liblinear", multi_class='ovr')
    
    def train(self, transcriptions, languages):
        """Gitt en rekke med IPA transkripsjoner og en rekke med språknavn, tren
        den logistisk regresjonsmodellen. De to rekkene må ha samme lendgen"""
        
        raise NotImplementedError()
    
    def predict(self, transcriptions):
        """Gitt en rekke med IPA transkripsjoner, finn ut det mest sansynnlige språket
        for hver transkripsjon. Rekken som returneres må ha samme lengden som rekken i input"""
        
        raise NotImplementedError()
        
    def _extract_unique_symbols(self, transcriptions, min_nb_occurrences=10):
        """Gitt en rekke med IPA fonetiske transkripsjoner, ektraher en liste med alle IPA 
        symboler som finnes i transkripsjonene og forekommer minst min_nb_occurrences."""
        raise NotImplementedError()
        
    def _extract_feats(self, transcriptions):
        """Gitt en rekke med IPA transkripsjoner, ekstraher en matrise av størrelse |T|x|F|,
        hvor |T| er antall transkripsjoner, og |F| er antall features brukt i modellen."""
        
        raise NotImplementedError()

    def evaluate(self, transcriptions, languages):  
        """Gitt en rekke med IPA transkripsjoner og en rekke med språknavn, evaluer hvor godt
        modellen fungerer ved å beregne:
        1) accuracy
        2) precision, recall og F1 for hvert språk
        3) micro- og macro-averaged F1.
        """
        
        # See API fra sklearn.metrics for å finne ut hvordan dette kan gjøres! 
        raise NotImplementedError()

## Trening 
Vi skal benytte oss av en logistisk regresjonsmodell, mer spesifikk klassen [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) fra `scikit-learn`. Men for å kunne bruke modellen må vi naturligvis først estimere parametrene basert på treningsdata. 
 Hva slags trekk (_features_) skal vi bruke i modellen vår? I denne oppgaven skal vi gjøre det enkelt for oss selv, og kun ta i betrakning forekomst av bestemte IPA-symboler som identifiserer ordlyder i den fonetiske transkripsjonen av ordet. _NB_: hvis du kan litt fonetikk fra før vil dere kanskje stusse på denne overforenklingen, da IPA-symboler brukes til å kode en god del andre egenskaper knyttet til talelyder slik som lengde, tone, trykk og intonasjon. Men i denne oppgaven skal vi gå den enkle veien og bruke alle symbolene i disse transkripsjonene uten å skille mellom ulike typer.

Trekkene vil ha binære verdier (1 hvis transkripsjonen inneholder symbolet og 0 ellers) og kan sees som en slags ``bag-of-sounds'', siden de vil fortelle oss hvilke ordlyder som forekommer i ordet, men ikke i hvilken rekkefølge.

__Opgave 1.1:__ Det første skrittet er å lage en liste over alle IPA-symboler som finnes i treningsettet. Metoden `_extract_unique_symbols` skal ta de fonetiske transkripsjonene fra treningssettet som input, og returnere en liste med alle fonetiske symboler (altså tegn) som finnes i disse transkripsjonene og forekommer minst 10 ganger. Implementer denne metoden. Antall symboler kan variere litt avhengig av den tilfeldige inndelingen mellom treningsettet og testsettet, men bør ligge på rundt 155 unike symboler.

In [24]:
def _extract_unique_symbols(self, transcriptions, min_nb_occurrences=10):
    """Gitt en rekke med IPA fonetiske transkripsjoner, ektraher en liste med alle IPA 
    symboler som finnes i transkripsjonene og forekommer minst min_nb_occurrences."""
    
    symbols = {}
    for word in transcriptions:
        for symbol in word:
            if symbol not in symbols:
                symbols[symbol] = 0
            symbols[symbol] += 1
    
    result = [symbol for symbol, n in symbols.items() if n > min_nb_occurrences]
    return result
    
# her kobler vi metoden vi har implementert til klassen
LanguageIdentifier._extract_unique_symbols = _extract_unique_symbols 



__Oppgave 1.2__: Deretter må vi implementere metoden `_extract_feats` som tar en liste fonetiske transkripsjoner og returnerer en matrise $X$ hvor hver rad tilsvarer en transkripsjon og hver kolonne representerer et bestemt trekk. La oss si vi har $m$ unike fonetiske symboler (ekstrahert med metoden ovenfor), og får en liste med $n$ fonetiske transkripsjoner $T = \{t_i \text{ hvor } 0 < i < n\}$. Metoden `_extract_feats(transcriptions)` må returnere en matrise $X$ av dimensjon $(n,m)$, hvor hver matrisecelle $X_{ij}$ er definert slik: 

$$
X_{ij}  =  \begin{cases} 1 \text{  hvis symbolet } j \text{ forekommer i transkripsjonen } t_i \\
0 \text{  ellers} \end{cases} 
$$

Tips: den enkleste fremgangsmåten kan være å starte med å lage en tom matrise slik som `X = np.zeros(n,m)` og deretter endre cellene hvor `X[i,j]` skal ha 1 som verdi i stedet for 0. 

In [25]:
import numpy as np

def _extract_feats(self, transcriptions, unique_symbols: list = []):
    """Gitt en rekke med IPA transkripsjoner, ekstraher en matrise av størrelse |T|x|F|,
    hvor |T| er antall transkripsjoner, og |F| er antall features brukt i modellen.
    Kan ta inn en liste symboler (brukes når man skal predicte)."""
    
    if len(unique_symbols) == 0:
        unique_symbols = _extract_unique_symbols(self, transcriptions)
    
    m = len(unique_symbols)
    n = len(transcriptions)
    X = np.zeros((n, m))
    
    for i in range(n):
        for j in range(m):
            if unique_symbols[j] not in transcriptions[i]:
                continue
            X[i][j] = 1
    
    return X

LanguageIdentifier._extract_feats = _extract_feats

__Oppgave 1.3__: Vi er nå klare til for å implementere funksjonen `train`. Metoden tar to lister som input, en liste fonetiske transkripsjoner og en liste med språknavn (de to listene må ha samme lengde). Metoden skal trene den logistiske regresjonsmodellen `self.model` ved å kalle `fit(X, y)`, hvor `X` er en matrise med alle trekk ekstrahert med metoden `_extract_feats`, og `y` er outputklassene. 

Merk at `scikit-learn` krever at outputklassene `y` må være en liste med heltall (og ikke strenger). Det betyr at dere må lage en mapping mellom språknavn og heltall (f.eks. ved å si at "norsk" er 0, "arabisk" er 1, "finsk" er 2, osv.). Når dere har både matrisen `X` og output `y` er det bare å kalle metoden `fit(X, y)` for å trene modellen. Trening kan ta noen minutter avhengig av maskinen deres.

In [26]:
def unique_languages(languages: list) -> list:
    """Gitt en rekke med språk, returnerer en rekke med unike språk."""
    
    unique_langs = []
    for l in languages:
        if l not in unique_langs:
            unique_langs.append(l)
    
    return unique_langs

def map_languages(languages: list, mapped: list = []) -> list:
    """Metode som tar inn en rekke med språk og mapper de til hvert sitt heltall. 
    Man kan også decode hvilket språk en liste heltall er ved å sende inn språklisten og listen med tall som parametre."""
    
    res = [x for x in languages]
    unique_langs = unique_languages(languages)
    
    for i, l in enumerate(res):
        res[i] = unique_langs.index(l)
    
    if len(mapped) == 0:
        return res
    
    if len(mapped) == 1:
        mapped = mapped[0]
        return unique_langs.index(mapped)
    
    return [unique_langs[x] for x in mapped]
    

In [27]:
def train(self: LanguageIdentifier, transcriptions: list, languages: list) -> None:
    """Gitt en rekke med IPA transkripsjoner og en rekke med språknavn, tren
    den logistisk regresjonsmodellen. De to rekkene må ha samme lengde."""
    
    X = _extract_feats(self, transcriptions)
    y = map_languages(languages)
    
    self.model.fit(X, y)

LanguageIdentifier.train = train

Vi er nå klare til å trene modellen (kan ta noen minutter):

In [28]:
model = LanguageIdentifier()
transcriptions = train_data.IPA.values
languages = train_data.språk.values
model.train(transcriptions, languages)


## Prediksjon og evaluering

Når modellen er trent kan vi anvende den på nye fonetiske transkripsjoner. 

__Oppgave 1.4__: Implementer metoden `predict`. Metoden tar som input en liste fonetiske transkripsjoner og predikerer det mest sannsynlige språket for hver transkripsjon. Listen som returneres må ha samme lengde som inputlisten. Husk at `scikit-learn` opererer med outputklasser representert som heltall, så dere må forvandle disse tallene tilbake til språknavnene.  

In [29]:
def predict(self: LanguageIdentifier, transcriptions: list) -> list:
    """Gitt en rekke med IPA transkripsjoner, finn ut det mest sansynnlige språket
    for hver transkripsjon. Rekken som returneres må ha samme lengden som rekken i input"""
    
    all_symbols = _extract_unique_symbols(self, train_data.IPA.values)
    feats = _extract_feats(self, transcriptions, all_symbols) # extracter feats med samme symboler som treningssettet så matrisen av feats blir riktig
    preds = self.model.predict(feats)
    preds = map_languages(languages, preds)
    return preds

LanguageIdentifier.predict = predict

Deretter kan dere se hvordan modellen fungerer i praksis:

In [30]:
# Vi kan nå teste modellen på nye data
predicted_langs = model.predict(["konstituˈθjon", "ɡrʉnlɔʋ", "stjourtnar̥skrauːɪn", "perusˌtuslɑki"])
print("Mest sansynnlige språk for ordene:", predicted_langs)

Mest sansynnlige språk for ordene: ['spansk', 'norsk', 'islandsk', 'finsk']


Det virker som om det er 50/50 om modellen predikerer norsk eller rumensk. Accuracy-en til modellen er likevel 93%, og den greier de andre språkene hver gang.


(Svarene bør være spansk, norsk, islandsk og finsk).

__Oppgave 1.5__: Til slutt kan vi gjennomføre en grundigere evaluering av modellen basert på testsettet. Implementer metoden `evaluate`. Metoden skal beregne og skrive ut de følgende evalueringsmålene:
* accuracy (den bør ligge rund 93 % hvis dere har gjort alt riktig.)
* precision, recall og $F_1$ for hvert språk
* micro- og macro-averaged F1.    
    
For å beregne disse stallene kan dere bruke metodene fra [`sklearn.metrics`](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

In [31]:
def evaluate(self: LanguageIdentifier, transcriptions: list, languages: list) -> None:  
    """Gitt en rekke med IPA transkripsjoner og en rekke med språknavn, evaluer hvor godt
    modellen fungerer ved å beregne:
    1) accuracy
    2) precision, recall og F1 for hvert språk
    3) micro- og macro-averaged F1.
    """
    
    prediction = model.predict(transcriptions)
    unique_langs = unique_languages(languages)
    
    accuracy = sklearn.metrics.accuracy_score(languages, prediction)
    print(f"Accuracy: {accuracy}\n")
    
    prec_rec = sklearn.metrics.precision_recall_fscore_support(languages, prediction)
    
    precision = [f"{unique_langs[i]}: {n}" for i,n in enumerate(prec_rec[0])] # list comprehension som legger til språk foran hver score -> ['kantonesisk: 0.9591753419065115', ...]
    print(f"Precision:\n{precision}\n")
    
    recall = [f"{unique_langs[i]}: {n}" for i,n in enumerate(prec_rec[1])]
    print(f"Recall:\n{recall}\n")
    
    f1score = sklearn.metrics.f1_score(languages, prediction, average=None)
    f1 = [f"{unique_langs[i]}: {n}" for i,n in enumerate(f1score)]
    print(f"F1:\n{f1}\n")
    
    f1_micro = sklearn.metrics.f1_score(languages, prediction, average='micro')
    print(f"F1 micro-averaged: {f1_micro}\n")
    
    f1_macro = sklearn.metrics.f1_score(languages, prediction, average='macro')
    print(f"F1 macro-averaged: {f1_macro}")
    
LanguageIdentifier.evaluate = evaluate
 
# Vi kan nå evaluere hvor godt modellen fungerer på testsett
model.evaluate(test_data.IPA.values, test_data.språk.values)

Accuracy: 0.9372315004635493

Precision:
['islandsk: 0.9636990468464814', 'fransk: 0.965181871231278', 'koreansk: 0.943609022556391', 'svensk: 0.9920031987205118', 'kantonesisk: 0.9604959025005253', 'swahilisk: 0.9492203118752499', 'arabisk: 0.9755395683453237', 'spansk: 0.9963197710079739', 'tysk: 0.9439655172413793', 'malayisk: 0.9989835332384631', 'vietnamesisk: 0.758985200845666', 'finsk: 0.950592885375494', 'rumensk: 0.9186550976138829', 'engelsk: 0.8522727272727273', 'japansk: 0.7672256653299308', 'mandarin: 0.9416683663063431', 'khmer: 0.9612555174104953', 'norsk: 0.8234086242299795', 'farsi: 0.9859183673469388', 'patwa: 0.9771993543179984']

Recall:
['islandsk: 0.9404314268751237', 'fransk: 0.9851101846337106', 'koreansk: 0.9261992619926199', 'svensk: 0.9947874899759422', 'kantonesisk: 0.9121931750149671', 'swahilisk: 0.9418766117833763', 'arabisk: 0.9530120481927711', 'spansk: 0.9763574433981166', 'tysk: 0.6422287390029325', 'malayisk: 0.9929278642149929', 'vietnamesisk: 0.738

## Analyse av modellen

Hva har modellen egentlig lært? En stor fordel med logistisk regresjon er at modellene er relativt enkle å tolke: Hvis en vekt $w_i$ i modellen har stor positiv verdi betyr det at sannsynnlighet for outputklassen _øker_ sammen med trekket $x_i$.  Likeledes betyr en negativ verdi at sannsynlighet for outputklassen _reduseres_ med større verdier av $x_i$. Og jo større vektet er (positiv eller negativ), jo større er effekten. 

Vi kan inspisere modellen vår for å finne ut hvilke språklyder som har størst effekt på prediksjonene. I `scikit-learn` er modellvektene lagret i variabelen `coef_` (merk underscoren ved slutten). Siden modellen vår er multiklasse (med 20 unike språk) og inneholder $m \approx 155$ trekk er vektene i `coef_` en matrise av dimensjon $(20,m)$. 

__Oppgave 1.6a__: Finn ut hvilket fonetisk symbol som bidrar mest til å øke sannsynnligheten for at et ord er klassifisert som norsk. Sjekk om det gir mening ved å telle hvor ofte symbolet forekommer i et norsk ord vs. ikke-norsk ord.

__Oppgave 1.6b__: Finn ut hvilket fonetisk symbol som bidrar mest til å redusere sannsynnligheten for at et ord er klassifisert som norsk.

Søk gjerne på nettet for å finnes ut hva disse fonetiske symbolene egentlig står for hvis du er interessert!


In [32]:
unique_symbols = model._extract_unique_symbols(transcriptions)

norsk_coef = model.model.coef_[map_languages(languages, ['norsk'])].tolist()
print(f"Oppgave 1.6a:\nSymbolet som øker sannsynligheten mest for at et ord er klassifisert som norsk: {unique_symbols[norsk_coef.index(max(norsk_coef))]}")
print()
print(f"Oppgave 1.6b:\nSymbolet som reduserer sannsynligheten mest for at et ord er klassifisert som norsk: {unique_symbols[norsk_coef.index(min(norsk_coef))]}")


Oppgave 1.6a:
Symbolet som øker sannsynligheten mest for at et ord er klassifisert som norsk: ʋ

Oppgave 1.6b:
Symbolet som reduserer sannsynligheten mest for at et ord er klassifisert som norsk: ²


# Del 2: sekvensmodellering

Vi skal nå jobbe på en viktig anvendelse av sekvensmodeller, nemlig å gjenkjenne navngitte entiteter (_Named Entity Recognition_ eller NER på engelsk). For å gjøre det så enkelt som mulig vil vi bruke en _Hidden Markov Model_ som sekvensmodell. Hvert ord skal assosieres med en bestemte klasse, og vi skal ta i bruk såkalt BIO-annotering (også kalt IOB i boken til Jurafsky og Martin) for å spesifisere hvilke som ord hører til en navngitt entitet. 

_NB_: I praksis er ikke HMM den mest hensiktsmessige sekvensmodellen for å gjenkjenne navngitte enheter. Andre modeller slik som _transformers_ vil være bedre egnet til denne oppgaven, med disse modellene er betydelige mer kompliserte å trene. 


### Data 
Vi skal trene modellen med dataene i `norne_train.txt`. Filen inneholder tokeniserte setninger (en per linje) hvor de navngitte entitetene er markert med XML-tags, som f.eks:

```xml
De første 43 minuttene hadde <ORG>Rosenborg</ORG> all makt og 
tilnærmet full kontroll på <LOC>Fredrikstad Stadion</LOC> .
```
I eksempelet over har vi 2 navngitte enheter, _Rosenborg_ (en organisasjon) og _Fredrikstad Stadion_ (et sted).

Vi har allerede implementert en funksjon `preprocess` som tar en tekst som input (som f.eks. setningene i trening- eller testsett) og ekstraherer lister over setninger og navngitte entiteter i disse setningene:

In [33]:
import oblig1b_utils
oblig1b_utils.preprocess("De første 43 minuttene hadde <ORG>Rosenborg</ORG> all makt og " +
                         "tilnærmet full kontroll på <LOC>Fredrikstad Stadion</LOC> .")

([['De',
   'første',
   '43',
   'minuttene',
   'hadde',
   'Rosenborg',
   'all',
   'makt',
   'og',
   'tilnærmet',
   'full',
   'kontroll',
   'på',
   'Fredrikstad',
   'Stadion',
   '.']],
 [[(5, 6, 'ORG'), (13, 15, 'LOC')]])

De navngitte entitetene er spesifisert som tupler $(i, j, tag)$ hvor $i$ er indeksen for starten av entiteten, $j$ er indeksen for slutten, og $tag$ er entitetstypen, som f.eks. ORG eller LOC. Indekstallene er på ordnivå:

### Klassen

Her er skjelettet for `NamedEntityRecogniser`-klassen  som skal implementeres:

In [34]:
import oblig1b_utils

class NamedEntityRecogniser:
    """Gjenkjenning av navngitte enheter ved bruk av HMM"""
    
    def __init__(self):
        """Intialiserer alle variablene som er nødvendig for å representere og 
        estimere  sekvensmodellen (en Hidden Markov Model) som brukes til å 
        gjenkjenne de navngitte enhetene"""
        
        # alle labellene som forekommer i treningsettet
        self.labels = set()

        # alle token som forekommer i treningsettet
        self.vocab = set()

        # hvor mange ganger en label (f.eks. B-ORG) forekommer i treningsettet
        self.label_counts = {}  

        # hvor mange overgang fra label_1 til label2 forekommer i treningsettet
        self.transition_counts = {}
        
        # hvor mange "utslipp" fra label til token forekommer i treningsettet
        # (Merk at vi legger et spesielt symbol for ord som aldri forekommer
        # i treningsettet, men kan forekomme i testsettet)
        self.emission_counts = {("O", "<UNK>"):1}
                
        # Sansynnlighet P(label_2 | label_1)
        self.transition_probs = {}
        
        # Sansynnlighet P(token | label)
        self.emission_probs = {}
    
    
    def fit(self, tagged_text):
        """Estimerer tallene og sansynnlighetene for HMM, basert på (tokenisert)
        tekst hvor navngitte enhetene er markert med XML tags (se norne.txt)"""
        
        # Ekstrahere setninger og navngitte enheter markert i hver setning
        sentences, all_spans = oblig1b_utils.preprocess(tagged_text)
                
        for sentence, spans in zip(sentences, all_spans):
            
            # Ekstrahere labelsekvenser, med BIO marking
            label_sequence = get_BIO_sequence(spans, len(sentence))
            
            # Oppdatere tallene 
            self._add_counts(sentence, label_sequence)
            
        # Beregne sansynnlighetene (transition og emission) ut fra tallene
        self._fill_probs()
        
    def _add_counts(self, sentence, label_sequence):
        """Oppdaterer variablene self.vocab, self.labels, self.label_counts, 
        self.transition_counts og  self.emission_counts, basert på setningen og 
        sekvenslabellen assosiert med dem. 
        Merk at setningen og label_sequence har samme lengde."""
        
        raise NotImplementedError()
        
    def _fill_probs(self, alpha_smoothing=1E-6):
        """Beregne sannsynlihetsfordelinger self.transition_probs og
        self.emission_probs basert på tallene som er samlet inn i 
        self.label_counts, self.transition_counts og self.emission_counts.
        
        Når det gjeler self.emission_probs bør vi legge Laplace smoothing, med en
        verdi for alpha som er alpha_smoothing."""
        
        raise NotImplementedError()
            
    
    def _beam_search(self, sentence):
        """Kjører beam-search på setningen (liste over tokens) og
        returnerer to outputs: 
        1) en labelsekvens (som har samme lengde som setningen)
        2) sansynnlighet for hele sekvensen """

        raise NotImplementedError()

                
    def label(self, text):
        """Gitt en tokenisert tekst, finner ut navngitte enheter og markere disse
        med XML tags. """
        sentences, _ = oblig1b_utils.preprocess(text)
        spans = []
        for sentence in sentences:
            sentence = [token if token in self.vocab else "<UNK>" for token in sentence]
            label_sequence, _ = self._beam_search(sentence)
            spans.append(oblig1b_utils.get_spans(label_sequence))
        
        return oblig1b_utils.postprocess(sentences, spans)

### BIO-markering


For å trene en HMM må hvert ord kobles til en merkelapp (_label_). En vanlig måte å gjøre dette er å bruke en såkalt BIO-markering, hvor hvert ord markeres som:
* 'O' (hvis ordet ikke tilhører en navngitt entitet)
* 'B-X' (hvis ordet er det første ordet i en navngitt entitet av type 'X')
* 'I-X' (hvis ordet tilhører en entitet av type 'X', men ikke er det første ordet)

__Oppgave 2.1__: Implementer funksjonen _get_BIO_sequence_ som tar som input en liste med ``text spans'' og setningslengden og gir tilbake en rekke (av samme lengde som setningen) med BIO-markeringer.

In [35]:
    
def get_BIO_sequence(spans: list, sentence_length: int) -> list:
    """Gitt en liste over "spans", representert som tuples (start, end, tag),
    og en setningslengde, produserer en sekvens med BIO (også kalt IOB) labeller
    for setningen. 
    Eksempel: hvis spans=[(1,3,'ORG')] og sentence_length=6 bør resultatet være
    ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O']"""
    
    result = ['O'] * sentence_length
    
    for span in spans:
        entity = span[2]
        current = [f"I-{entity}"] * (span[1] - span[0])
        current[0] = f"B-{entity}"
        
        for i, l in enumerate(current):
            result[i + span[0]] = l
    
    return result


### Telling

Som vi så i forelesningen er Hidden Markov Models definert med en vokabular (som tilsvarer ``observasjonene''), et sett mulig merkelapp (de skjulte tilstandene), og to sannsynlighetsfordelinger:
* Den første fordelingen er kalt transisjonsmodell og definert som $P(label_t | label_{t-1})$. Transisjonsmodellen forteller oss hvor sannsynlig det er at $label_{t-1}$ (assosiert med ord $w_{t-1}$) følges av $label_t$ (assosiert med ord $w_t$). 
* Den andre fordelingen er emisjonsmodellen, definert som $P(w_t | label_t)$. Emisjonsmodellen forteller oss hvor sannsynlig det er å observere ordet $w_t$ hvis merkelappen for dette ordet er $label_t$.

For å estimere disse to sannsynlighetsfordelinger må vi telle:
* Alle ordene som forekommer i treningssettet
* Alle BIO-labels som forekommer i treningssettet
* Antall ganger hver BIO-label forekommer i treningssettet
* Antall ganger to BIO-labels følger hverandre i treningssettet
* Antall ganger et ord er observert med en BIO-label i treningssettet

__Oppgave 2.2__: Implementer metoden `_add_counts` som oppdaterer variablene som inneholder disse tallene.  

In [36]:
def _add_counts(self: NamedEntityRecogniser, sentence: list, label_sequence: list) -> None:
    """Oppdaterer variablene self.vocab, self.labels, self.label_counts, 
    self.transition_counts og  self.emission_counts, basert på setningen og 
    sekvenslabellene assosiert med dem. 
    Merk at setningen og label_sequence må ha samme lengde."""
    
    # print(f"Sentence: {sentence}")
    # print(f"Label sequence: {label_sequence}")
    
    prev_label = None
    for token, label in zip(sentence, label_sequence):        
        # self.vocab (alle ordene som forekommer)
        if token not in self.vocab:
            self.vocab.add(token)
        
        # self.labels (alle bio-labels som forekommer)
        if label not in self.labels:
            self.labels.add(label)
        
        # self.label_counts (antall ganger hver bio-label forekommer)
        if not self.label_counts.get(label):
            self.label_counts[label] = 0
        self.label_counts[label] += 1
        
        # self.transition_counts (antall ganger to bio-labels følger hverandre)
        if prev_label:
            trans = (prev_label, label)
            if not self.transition_counts.get(trans):
                self.transition_counts[trans] = 0
            self.transition_counts[trans] += 1
        prev_label = label
        
        # self.emission_counts (antall ganger et ord er observert med en bio-label)
        emission = (label, token)
        if not self.emission_counts.get(emission):
            self.emission_counts[emission] = 0
        self.emission_counts[emission] += 1
    
NamedEntityRecogniser._add_counts = _add_counts

### Sannsynligheterfordelinger

Med hjelp av disse tallene kan vi nå estimere transisjonsmodellen og emisjonsmodellen. For emisjonsmodellen bør dere legge til _Laplace smoothing_ for å gjøre modellen mer robust, da treningssettet er relativt lite. La oss si at $C(label)$ er antall ganger BIO-merkelappen $label$ ble observert, og $C(label, token)$ antall ganger ordet $token$ ble observert sammen med $label$.  Ved hjelp av Laplace smoothing definerer vi sannsynligheten $P(token | label)$ slik:

$$
P(token | label) = \frac{C(label, token) + \alpha}{C(label) + \alpha V}
$$

hvor $V$ er størrelsen på vokabularet. 

__Oppgave 2.3__: Implementer metoden `_fill_probs` som beregner transisjonsmodellen og emisjonsmodellen.  

In [37]:
def _fill_probs(self: NamedEntityRecogniser, alpha_smoothing=1E-6):
    """Beregne sannsynlihetsfordelinger self.transition_probs og
    self.emission_probs basert på tallene som er samlet inn i 
    self.label_counts, self.transition_counts og self.emission_counts.
    
    Når det gjeler self.emission_probs bør vi legge Laplace smoothing, med en
    verdi for alpha som er alpha_smoothing."""
    
    # emission_probs:
    for key, count in self.emission_counts.items():
        p = (count + alpha_smoothing) / (self.label_counts[key[0]] + (alpha_smoothing * len(self.vocab))) 
        self.emission_probs[key] = p
    
    # transition_probs:
    for key, count in self.transition_counts.items():
        p = count / self.label_counts[key[0]] # C(label1,label2) / C(label1)
        self.transition_probs[key] = p
    
NamedEntityRecogniser._fill_probs = _fill_probs

Og vi kan nå trene modellen:

In [38]:
# Creating and training the model
with open("norne_train.txt") as fd:
    model = NamedEntityRecogniser()
    training_texts = fd.read()
    model.fit(training_texts)


### Dekoding med beam search

Til slutt skal vi implementere metoden `_beam_search` som skal finne den mest sannsynlige label-sekvensen for en setning ved hjelp av beam-search-algoritmen. 

Vi har sett hvordan beam search fungerer i forelesningen. Kort fortalt består et _beam_ av en mengde _hypoteser_, hvor en hypotese representerer her en mulig sekvens av BIO-labeller for setningen. Vi bygger opp disse hypotesene ord etter ord, inntil alle ordene er behandlet. I beam-search begrenser vi antall hypoteser som kan lagres etter hvert steg (=_beam width_) for å unngå en kombinatorisk eksplosjon.

__Oppgave 2.4__: Implementer metoden `_beam_search`. For å gjøre det litt lettere er metoden allerede delvis implementert.

In [39]:
def _beam_search(self: NamedEntityRecogniser, sentence: list, beam_width=3):
    """Kjører beam search på setningen (liste over tokens), og
    returnerer to outputs: 
    1) en labelsekvens (som har samme lengde som setningen)
    2) sansynnlighet for hele sekvensen """

    # Vi starter med en beam som består av én tom hypotese
    # Hver hypotese i beam består av to elementer:
    # -'labels' er en liste over BIO-labels (en label per ord i setningen)
    # -'prob' er sannsynligheten for denne labelsekvensen
    beam = [{"labels":[], "prob":1.0}]
    
    for token in sentence:
        new_hypotheses = []

        for hypothesis in beam:
            if hypothesis["labels"]:
                prev_label = hypothesis["labels"][-1]
            else:
                prev_label = None

            for label in self.labels:
                # emission prob av current token gitt current label. hvis den ikke finnes, bruk UNK emission prob
                emission_prob = self.emission_probs.get(
                    (label, token),
                    self.emission_probs.get((label, '<UNK>'), 0)
                )

                transition_prob = self.transition_probs.get(
                    (prev_label, label),
                    1.0 if prev_label is None else 0
                )
                
                new_prob = hypothesis["prob"] * transition_prob * emission_prob
                
                new_hypothesis = {"labels": hypothesis["labels"][:] + [label], "prob": new_prob}
                new_hypotheses.append(new_hypothesis)

        # behold beam_width beste hypoteser
        beam = sorted(new_hypotheses, key=lambda x: x["prob"], reverse=True)[:beam_width]  
        
    # Vi velger hypotesen med høyest sannsynlighet
    best_hypothesis = sorted(beam, key=lambda x : x["prob"])[-1]

    # Og returnerer labelsekvensen og dens sannsynlighet
    return best_hypothesis["labels"], best_hypothesis["prob"]

NamedEntityRecogniser._beam_search = _beam_search            

Når dere har implementert modellen ferdig kan dere teste ut hvordan den fungerer ved å kalle metoden `label`:

In [40]:
# Applying the model to a sample sentence
model.label("Kjell Magne Bondevik var statsminister i Norge .")
# Forventet svar: '<PER>Kjell Magne Bondevik</PER> var statsminister i <GPE>Norge</GPE> .'

'<PER>Kjell Magne Bondevik</PER> var statsminister i <GPE>Norge</GPE> .'

### Valgfritt spørsmål (for de modigste)
 
Det er ofte en dårlig idé å gange mange små sannsynligheter, da vi kan havne i numerisk _underflow_. En vanlig løsning er å benytte log-sannsynligheter som kan summeres i stedet for å multipliseres, slik som forklart [her](http://www.cs.columbia.edu/~mcollins/cs4705-fall2018/notes-on-logs.pdf).

__Oppgave 2.5__: Prøv å implementere beam search algoritmen med log-sannsynligheter.  